| [![back](../../media/navigation/back.png)](../../exercises/tissue-classification-segmentation/ex4.html) | [![home](../../media/navigation/home.png)](../../index.html) | [![next](../../media/navigation/next.png)](../../exercises/tissue-classification-segmentation/ex6.html) |
| :---: | :---: | :---: |
| Ex 4.4: Semi-automatic segmentation using SAM | • | Ex 4.6: Area of signal per slide |

# 4. Tissue segmentation and classification

## 4.5 Segmentation quality control

Are you happy with the segmentation that you made at the previous exercise? Or at the opposite, do you think that it is quite terrible? These descriptions are very arbitrary and depend on who is talking.

In this exercise, we focus only on **segmentation** quality control.

### 4.5.1 Segmentation metrics (reference vs prediction)

#### Goals:

- Build a clean validation set with `GT_*` (reference imported with `unpack-annotations.groovy`) and `Pred_*` (predicted) annotations.
- Choose one segmentation workflow among [Ex 4.1](../../exercises/tissue-classification-segmentation/ex1.html), [Ex 4.2](../../exercises/tissue-classification-segmentation/ex2.html), [Ex 4.3](../../exercises/tissue-classification-segmentation/ex3.html) or [Ex 4.4](../../exercises/tissue-classification-segmentation/ex4.html).
- Use QuPath to generate `Inter_Target` (`GT ∩ Pred`) and `Union_Target` (`GT ∪ Pred`).
- Export geometric areas (`A_GT`, `A_Pred`, `A_Inter`, `A_Union`) and compute the metrics yourself in a spreadsheet (`IoU`, `Dice`, `Precision`, `Recall`).

#### Suggested examples:

| Workflow | Suggested target class | Source notebook |
| --- | --- | --- |
| Fully manual segmentation | `Glomerulus` or `Kidney` | [Ex 4.1](../../exercises/tissue-classification-segmentation/ex1.html) |
| Threshold-based segmentation | `Lung` | [Ex 4.2](../../exercises/tissue-classification-segmentation/ex2.html) |
| N-classes pixel classifier | `Epithelium` | [Ex 4.3](../../exercises/tissue-classification-segmentation/ex3.html) |
| SAM segmentation | `Glomerulus` or `Kidney` | [Ex 4.4](../../exercises/tissue-classification-segmentation/ex4.html) |

### A. Prepare the validation project in QuPath

- Choose one workflow to evaluate (`Ex 4.1`, `Ex 4.2`, `Ex 4.3` or `Ex 4.4`) and one target class.
- Start from a copy of the corresponding project and keep only validation images (images that were not used to tune your segmentation).
- Save your project (![Kenney-Ctrl](../../media/inputs-icons/keyboard_ctrl.svg)+![Kenney-S](../../media/inputs-icons/keyboard_s.svg)).
- Keep/create one prediction class for your chosen target: `Pred_Target`.
- Open the top-bar menu "Automate" > "Show script editor".
- In the script editor, open [scripts/tissue-segmentation-classification/unpack-annotations.groovy](../../scripts/tissue-segmentation-classification/unpack-annotations.groovy).
- Click "Run" for one image to check that reference annotations are imported correctly.
- Use ![QP more option](../../media/qp-icons/more-options.png) "more options" next to "Run" and choose "Run for project" to import GT for all validation images.
- Keep the class name imported by `unpack-annotations.groovy` (for example `Glomerulus`) and reuse this exact name as `gt_class` in step B.
- If one biological structure is fragmented into several polygons, merge these polygons before continuing.
- If polygons correspond to different structures or different positions, do **not** merge them.
- Save again (![Kenney-Ctrl](../../media/inputs-icons/keyboard_ctrl.svg)+![Kenney-S](../../media/inputs-icons/keyboard_s.svg)).

**Warning**

Do not evaluate on images used to train/tune your model. Mixing train and validation images leads to over-optimistic metrics.

`unpack-annotations.groovy` requires the expected annotation files (JSON and `what.txt`) in the dataset structure. If they are missing, no GT will be imported.

### B. Create `Inter_Target` and `Union_Target` in QuPath

#### a. Test on one image

- Open the top-bar menu "Automate" > "Show script editor".
- In the script editor, open [scripts/tissue-segmentation-classification/make-intersection-union.groovy](../../scripts/tissue-segmentation-classification/make-intersection-union.groovy).
- Edit only the 4 class names at the top of the script so they match your project:

```groovy
def gt_class = "Glomerulus"  // class imported by unpack-annotations.groovy
def pred_class = "Pred_Target"
def inter_class = "Inter_Target"
def union_class = "Union_Target"
```

- Click "Run" for the current image.
- In the annotations list, check that `Inter_Target` and `Union_Target` were created.

#### b. Apply to the whole project

- Use ![QP more option](../../media/qp-icons/more-options.png) "more options" next to "Run" and choose "Run for project".
- Save your project (![Kenney-Ctrl](../../media/inputs-icons/keyboard_ctrl.svg)+![Kenney-S](../../media/inputs-icons/keyboard_s.svg)).

### C. Export measurements and compute metrics

#### a. Export measurements from QuPath

- Save your project (![Kenney-Ctrl](../../media/inputs-icons/keyboard_ctrl.svg)+![Kenney-S](../../media/inputs-icons/keyboard_s.svg)).
- In the top-bar menu, go to "Measure" > "Export measurements...".
- Move all validation images to the right column.
- Choose a path for your TSV file (for example `measurements.tsv`).
- Set "Export type" to `Annotations`, then export.

#### b. Compute metrics in LibreOffice Calc / Excel

- Keep rows for the 4 classes chosen in step B: `gt_class`, `pred_class`, `inter_class`, `union_class`.
- Build one summary line per image with:
    - `A_GT` = area of `gt_class`
    - `A_Pred` = area of `pred_class`
    - `A_Inter` = area of `inter_class`
    - `A_Union` = area of `union_class`
- If `A_Union` is missing: `A_Union = A_GT + A_Pred - A_Inter`.
- Compute:
    - `TP = A_Inter`
    - `FP = A_Pred - A_Inter`
    - `FN = A_GT - A_Inter`
    - `IoU = A_Inter / A_Union`
    - `Dice = 2*A_Inter / (A_GT + A_Pred)`
    - `Precision = TP / (TP + FP)`
    - `Recall = TP / (TP + FN)`

**Tip**

To compare methods fairly (`Ex 4.1` vs `Ex 4.2` vs `Ex 4.3` vs `Ex 4.4`), keep the same validation images and the same ground-truth annotations, then compare the median `IoU` and `Dice`.

**Note**

In this chapter, we only cover segmentation quality control. Classification quality control will be covered later in the object classification chapter.